In [1]:
import findspark

findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, transform

In [3]:
spark = SparkSession.builder.appName("MoviesAnalysis") \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()

23/08/17 05:27:27 WARN Utils: Your hostname, SeongGils-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.219.132 instead (on interface en0)
23/08/17 05:27:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/17 05:27:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/17 05:27:30 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:
movie_data = spark \
    .read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv("hdfs://localhost:9000/data/movies/KC_KOBIS_BOX_OFFIC_MOVIE_INFO_202105.csv")


In [10]:
movie_data.head()

Row(SN=1, MOVIE_NM='분노의 질주: 더 얼티메이트', MNG_NM='저스틴 린', MAKR_NM=None, IMPORT_CMPNY_NM='유니버설픽쳐스인터내셔널 코리아(유)', DISTB_CMPNY_NM='유니버설픽쳐스인터내셔널 코리아(유)', OPEN_DE=20210519, MOVIE_TY_NM='개봉영화', MOVIE_STLE_NM='장편', NLTY_NM='미국', WNTY_SCREEN_CO=2296, WNTY_SELNG_AM=17268076580, WNTY_AUDE_CO=1790155, SU_SELNG_AM=4197011990, SU_AUDE_CO=423112, GENRE_NM='액션', GRAD_NM='12세이상관람가', MOVIE_SE='일반영화')

In [12]:
### 함수를 통한 실행계획과 SQL의 실행 계획 차이 비교

# csv -> View
movie_data.createOrReplaceTempView("movie_data")

In [16]:
# 함수를 통한 호출 실행계획
movie_data.groupBy("NLTY_NM") \
    .count() \
    .explain()

## daptiveSparkPlan isFinalPlan=false:
# 이 부분은 적응형 실행 계획의 시작
# 적응형 실행 계획은 실행 중에 최적의 처리 경로를 선택할 수 있는 기능을 제공합니다. isFinalPlan=false는 이 계획이 아직 최종적인 것이 아니라는 것

## HashAggregate(keys=[NLTY_NM#79], functions=[count(1)]):
# 데이터를 그룹화, 집계 함수를 적용 group by NLTY_NM, count() 중간 결과로 활용

## Exchange hashpartitioning(NLTY_NM#79, 200), ENSURE_REQUIREMENTS, [plan_id=86]:
# 데이터의 분산처리를 위해 데이터를 교환하는 단계,  'NLTY_NM' 열을 기준으로 해시 파티셔닝을 수행하며, 200개의 파티션으로 데이터를 분산
# ENSURE_REQUIREMENTS는 실행 환경의 요구 사항을 충족시키기 위한 추가 작업을 수행

## ashAggregate(keys=[NLTY_NM#79], functions=[partial_count(1)]):
# 이 단계는 이전 단계에서 분산된 데이터를 다시 그룹화하고 집계 함수를 적용
# partial_count(1) 집계 함수를 사용하여 각 그룹 내의 레코드 수를 세는 작업을 수행합니다. 'NLTY_NM' 열을 기준으로 그룹화


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[NLTY_NM#79], functions=[count(1)])
   +- Exchange hashpartitioning(NLTY_NM#79, 200), ENSURE_REQUIREMENTS, [plan_id=125]
      +- HashAggregate(keys=[NLTY_NM#79], functions=[partial_count(1)])
         +- FileScan csv [NLTY_NM#79] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/data/movies/KC_KOBIS_BOX_OFFIC_MOVIE_INFO_202105..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<NLTY_NM:string>


In [17]:
# Query를 통한 호출 실행계획
spark.sql("""
SELECT NLTY_NM, count(1)
from movie_data as t1 group by t1.NLTY_NM 
""").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[NLTY_NM#79], functions=[count(1)])
   +- Exchange hashpartitioning(NLTY_NM#79, 200), ENSURE_REQUIREMENTS, [plan_id=138]
      +- HashAggregate(keys=[NLTY_NM#79], functions=[partial_count(1)])
         +- FileScan csv [NLTY_NM#79] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/data/movies/KC_KOBIS_BOX_OFFIC_MOVIE_INFO_202105..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<NLTY_NM:string>


In [23]:
q = """
select t1.NLTY_NM,
sum(t1.WNTY_SCREEN_CO) as screen_total
from movie_data as t1
group by t1.NLTY_NM
order by screen_total desc
limit 5
"""
spark.sql(q).show()

+-------+------------+
|NLTY_NM|screen_total|
+-------+------------+
|   미국|       14242|
|   한국|       12075|
|   일본|        3169|
|   영국|        1003|
| 헝가리|         691|
+-------+------------+


In [26]:
spark.sql(q).explain()

### dataframe으로 sql 실행 시 실행 계획
# csv -(read)> dataframe -(group by)> dataframe -(sum)> dataframe -(change column name)> dataframe -(sort)> dataframe -(limit)> dataframe -(collect)> Array()  

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[screen_total#422L DESC NULLS LAST], output=[NLTY_NM#79,screen_total#422L])
   +- HashAggregate(keys=[NLTY_NM#79], functions=[sum(WNTY_SCREEN_CO#80)])
      +- Exchange hashpartitioning(NLTY_NM#79, 200), ENSURE_REQUIREMENTS, [plan_id=445]
         +- HashAggregate(keys=[NLTY_NM#79], functions=[partial_sum(WNTY_SCREEN_CO#80)])
            +- FileScan csv [NLTY_NM#79,WNTY_SCREEN_CO#80] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/data/movies/KC_KOBIS_BOX_OFFIC_MOVIE_INFO_202105..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<NLTY_NM:string,WNTY_SCREEN_CO:int>


In [31]:
type(spark.sql(q).collect())

## before show() : 이전까지 트렌스포메이션은 하나 또는 그룹의 데이터 프레임을 반환
## after show() : 이후에는 이제까지 나온 Dataframe을 사용하는 언어ㅔ 맞게 array or list로 변환

list

In [34]:
### 타입형/비타입형 API의 개념과 차이점
### 스파크가 구조적 API의 데이터 흐름을 해석, 클러스터 실행 방식

## 구조적 API에는 비타입형(Dataframe)과 타입형(DataSet)으로 구분
## 이 차이는 스키마(data 구성요소)에 명시된 데이터 타입의 일치여부를 언제 확인하는지에 따라 구분됨

# Dataset : 컴파일타임에서 데이터 요소와 데이터 타입의 일치 여부를 확인,
# 따라서 명시적으로 데이터타입을 선언하는 JVM 기반 언어에서만 사용 가능

# Dataframe : Row 타입으로 구성된 Dataset

### Row타입이란?
# 스파크가 사용하는 "연산에 최적화된 인메모리 포맷"의 내부적인 표현방식
# Row타입 사용시 가비지 컬렉션과 객체 초기화 부하가 있는 JVM 데이터 타입을 사용하는 대신 자체 데이터
# 포맷을 사용하여
# 결론적으로 Dataframe 사용시 매우 효율적인 연산이 가능

In [35]:
## 구조적 API의 실행 과정
# 1. DataFrame/Dataset/SQL을 통해 코드 작성
#2. 정상적인 코드라면 스파크(카탈리스트 옵티마이저)가 논리적 실행계획으로 변환
# 3. 스파크는 논리적 실행 계획을 물리적 실행 계획으로 변환: 이때, 추가적인 최적화 방법이 있는지 확인
# 4. 스파크는 클러스터에서 물리적 실행 계획(RDD 처리)을 실행합니다.

In [36]:
## 논리적 실행 계획 : 논리적 실행계획 단계에서는 추상적 트랜스포메이션만 표현(드라이버나 익스큐터의 정보는 고려X)


# 검증 전 논리적 실행 계획 : 사용자가 작성한 코드를 유효성과 테이블의 컬럼 존재 여부만 판단하여 논리적 실행 계획으로 변환 (컴파일과 같은 내용)
# 검증된 논리적 실행 계획 : 카탈로그, 모든 테이블의 저장소 그리고 DataFrame 정보등 실제 데이터를 활용하여 스파크 분석기(analyzer)가 검증한 논리적 실행 계획
# 최적화된 논리적 실행 계획 : 검증된 논리적 실행계획을 Catalyst Optimizer가 조건절 푸쉬 다운이나 선택절 구문을 이용해 논리적 실행 계획을 최적화함

In [40]:
## 물리적 실행 계획 : 논리적 실행 계획을 클러스트 환경에서 실행하는 방법을 정의
# 다양한 물리적 계획 중 비용 모델을 이용하여 비교 후 최적의 전략을 클러스터에 전달, 실행

In [41]:
### 구조적 API중 집계연산

In [44]:
df = spark.read.csv("hdfs://localhost:9000/data/movies/KC_KOBIS_BOX_OFFIC_MOVIE_INFO_202105.csv", header=True)

In [45]:
df.show()

+---+--------------------------------+-----------------+-----------------------------------+-----------------------------------+-------------------------------------+--------+-----------+-------------+-------+--------------+-------------+------------+-----------+----------+----------+--------------+-------------+
| SN|                        MOVIE_NM|           MNG_NM|                            MAKR_NM|                    IMPORT_CMPNY_NM|                       DISTB_CMPNY_NM| OPEN_DE|MOVIE_TY_NM|MOVIE_STLE_NM|NLTY_NM|WNTY_SCREEN_CO|WNTY_SELNG_AM|WNTY_AUDE_CO|SU_SELNG_AM|SU_AUDE_CO|  GENRE_NM|       GRAD_NM|     MOVIE_SE|
+---+--------------------------------+-----------------+-----------------------------------+-----------------------------------+-------------------------------------+--------+-----------+-------------+-------+--------------+-------------+------------+-----------+----------+----------+--------------+-------------+
|  1|      분노의 질주: 더 얼티메이트|        저스틴 린|              

In [48]:
# count : 기존 count는 액션이였지만, 해당 count는 트랜스포메이션

from pyspark.sql.functions import count

df.select(count("SN").name("SN COUNT")).show()

+--------+
|SN COUNT|
+--------+
|     214|
+--------+


In [53]:
# approx_count_distinct : 정확한 count값이 필요하지 않다면 최대 추정 오류율(maximum estimation error) 설정 후 조회 시 근사한 오차를 가지는 개수를 출력

from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct("SN", 0.2)).show()

+-------------------------+
|approx_count_distinct(SN)|
+-------------------------+
|                      245|
+-------------------------+


In [58]:
## 분산과 표준편차

#분산과 표준편차를 구할 수 있는 함수
# 분산과 표준편차는 평균 주변에 데이터가 분포된 정도를 측정하는 방법
# samp : 표본(sample) / 그중 부분 (전국 고등학생 키)
# pop : 모(population) / 진짜 
# stddev : 표준편차(standard deviation)
# var : 표준분산(standard variance)
#모집단 : 진짜 전국의 키
# 표본집단 : 서울 100명, 부산 100명, 대구 100명 .... 구마다 100명씩 뽑아본다.

from pyspark.sql.functions import var_pop, stddev_pop, var_samp, stddev_samp

(df.select(var_pop("WNTY_SELNG_AM"),
           var_samp("WNTY_SELNG_AM"),
           stddev_pop("WNTY_SELNG_AM"),
           stddev_samp("WNTY_SELNG_AM"))
 .show())

+----------------------+-----------------------+-------------------------+--------------------------+
|var_pop(WNTY_SELNG_AM)|var_samp(WNTY_SELNG_AM)|stddev_pop(WNTY_SELNG_AM)|stddev_samp(WNTY_SELNG_AM)|
+----------------------+-----------------------+-------------------------+--------------------------+
|  1.540661170504616...|   1.547894321539849...|     1.2412337292003534E9|        1.24414401157577E9|
+----------------------+-----------------------+-------------------------+--------------------------+


In [57]:
df.select("WNTY_SELNG_AM").show()

+-------------+
|WNTY_SELNG_AM|
+-------------+
|  17268076580|
|   3038945310|
|   2945572300|
|   2329329840|
|   1957213560|
|   2044649290|
|   1299467860|
|   1278892690|
|   1232214520|
|    839827490|
|    717693410|
|    653801130|
|    644299040|
|    511060350|
|    353578540|
|    220196200|
|    202838760|
|    199479620|
|    155307470|
|    172229430|
+-------------+


In [59]:
## skewness(비대칭도,왜도), kurtosis(첨도)

# 데이터의 변곡점 측정 방법
# 확률변수의 확률분포로 데이터 모델링할 때 필요

from pyspark.sql.functions import skewness, kurtosis

df.select(skewness("WNTY_SELNG_AM"), kurtosis("WNTY_SELNG_AM")).show()

+-----------------------+-----------------------+
|skewness(WNTY_SELNG_AM)|kurtosis(WNTY_SELNG_AM)|
+-----------------------+-----------------------+
|     12.347888659269193|     164.93203214282292|
+-----------------------+-----------------------+


In [62]:
## corr(상관관계)과 covar(공분산)
# 두 컬럼값 사잉의 영향도를 비교하는 값
# 상관관계는 피어슨 상관계수(-1 < 상관계수 < 1)
from pyspark.sql.functions import corr, covar_samp, covar_pop

# WNTY_SELNG_AM 매출금액 WNTY_AUDE_CO 관람인원
df.select(corr("WNTY_SELNG_AM", "WNTY_AUDE_CO"),
          covar_samp("WNTY_SELNG_AM", "WNTY_AUDE_CO"),
          covar_pop("WNTY_SELNG_AM", "WNTY_AUDE_CO")
          ).show()

+---------------------------------+---------------------------------------+--------------------------------------+
|corr(WNTY_SELNG_AM, WNTY_AUDE_CO)|covar_samp(WNTY_SELNG_AM, WNTY_AUDE_CO)|covar_pop(WNTY_SELNG_AM, WNTY_AUDE_CO)|
+---------------------------------+---------------------------------------+--------------------------------------+
|               0.9998467030541379|                   1.610468908225247E14|                  1.602943352579334...|
+---------------------------------+---------------------------------------+--------------------------------------+


In [63]:
from pyspark.sql.functions import max, dense_rank, rank, col
from pyspark.sql.functions import desc
from pyspark.sql.window import Window

windowSpec = Window \
    .partitionBy("NLTY_NM") \
    .orderBy(desc("WNTY_SCREEN_CO")) \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

maxWNTY_SCREEN_CO = max(col("WNTY_SCREEN_CO")).over(windowSpec)
WNTY_SCREEN_CODenseRank = dense_rank().over(windowSpec)
WNTY_SCREEN_CORank = rank().over(windowSpec)

df.where("NLTY_NM IS NOT NULL").orderBy("NLTY_NM") \
    .select(col("NLTY_NM"), col("WNTY_SCREEN_CO"), maxWNTY_SCREEN_CO.alias("Max"),
            WNTY_SCREEN_CODenseRank.alias("denseRank"), WNTY_SCREEN_CORank.alias("Rank")).show()

+--------+--------------+---+---------+----+
| NLTY_NM|WNTY_SCREEN_CO|Max|denseRank|Rank|
+--------+--------------+---+---------+----+
|    기타|            85| 85|        1|   1|
|    기타|            70| 85|        2|   2|
|    기타|             7| 85|        3|   3|
|    기타|            44| 85|        4|   4|
|    기타|           223| 85|        5|   5|
|    기타|            14| 85|        6|   6|
|노르웨이|            63| 63|        1|   1|
|    독일|             7|  7|        1|   1|
|    독일|            50|  7|        2|   2|
|  러시아|             1|  1|        1|   1|
|    미국|           780|780|        1|   1|
|    미국|           710|780|        2|   2|
|    미국|            71|780|        3|   3|
|    미국|            70|780|        4|   4|
|    미국|           669|780|        5|   5|
|    미국|            53|780|        6|   6|
|    미국|           505|780|        7|   7|
|    미국|            50|780|        8|   8|
|    미국|           387|780|        9|   9|
|    미국|            38|780|       10|  10|
+-------

In [67]:
## Join

person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100])
    , (1, "one Tak", 0, [500, 250, 100])
    , (2, "two Taks", 0, [250, 100])
]).toDF("id", "name", "graduate_program", "spark_status")

graduate_program = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley")
    , (1, "Masters", "EECS", "UC Berkeley")
    , (1, "Ph.D", "EECS", "UC Berkeley")
]).toDF("id", "degree", "department", "school")

spark_status = spark.createDataFrame([
    (500, "Vice President")
    , (250, "PMC Member")
    , (100, "Contributor")
]).toDF("id", "status")

In [68]:
person.show()
graduate_program.show()
spark_status.show()

+---+-------------+----------------+---------------+
| id|         name|graduate_program|   spark_status|
+---+-------------+----------------+---------------+
|  0|Bill Chambers|               0|          [100]|
|  1|      one Tak|               0|[500, 250, 100]|
|  2|     two Taks|               0|     [250, 100]|
+---+-------------+----------------+---------------+
+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|Masters|                EECS|UC Berkeley|
|  1|   Ph.D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+
+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+


In [69]:
person.createOrReplaceTempView("person")
graduate_program.createOrReplaceTempView("graduateProgram")
spark_status.createOrReplaceTempView("sparkStatus")

In [72]:
joinExpression = person["graduate_program"] == graduate_program["id"]

person.join(graduate_program, joinExpression).show()

+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|      one Tak|               0|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  2|     two Taks|               0|     [250, 100]|  0|Masters|School of Informa...|UC Berkeley|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+


In [75]:
person.join(graduate_program, person["graduate_program"] == graduate_program["id"]).orderBy(person["id"]).show()

+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|      one Tak|               0|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  2|     two Taks|               0|     [250, 100]|  0|Masters|School of Informa...|UC Berkeley|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+


In [76]:
person.join(graduate_program, joinExpression, "left").show()

+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  1|      one Tak|               0|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  2|     two Taks|               0|     [250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+


In [77]:
## 왼쪽 세미조인 : 중복키값 포함 왼쪽 세미 조인 / 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우 키가 일치하는 왼쪽 데이터셋만 유지

graduate_program.join(person, joinExpression, "left_semi").show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
+---+-------+--------------------+-----------+


In [80]:
from pyspark.sql.functions import expr

# bool 형식으로 표현 가능한 모든 조건신은 가능함

person.withColumnRenamed("id", "personId") \
    .join(spark_status, expr("array_contains(spark_status,id)")).show()

+--------+-------------+----------------+---------------+---+--------------+
|personId|         name|graduate_program|   spark_status| id|        status|
+--------+-------------+----------------+---------------+---+--------------+
|       0|Bill Chambers|               0|          [100]|100|   Contributor|
|       1|      one Tak|               0|[500, 250, 100]|500|Vice President|
|       1|      one Tak|               0|[500, 250, 100]|250|    PMC Member|
|       1|      one Tak|               0|[500, 250, 100]|100|   Contributor|
|       2|     two Taks|               0|     [250, 100]|250|    PMC Member|
|       2|     two Taks|               0|     [250, 100]|100|   Contributor|
+--------+-------------+----------------+---------------+---+--------------+


In [81]:
## 중복 컬럼명 처리
# join할때 중복된 컬럼명으로 select할시 애매모호한 컬럼명이라고 에러 출력

gradProgramDupe = graduate_program.withColumnRenamed("id", "graduate_program")

joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]
person.join(gradProgramDupe, joinExpr).show()
person.join(gradProgramDupe, joinExpr).select("graduate_program").show()

+---+-------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+-------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  1|      one Tak|               0|[500, 250, 100]|               0|Masters|School of Informa...|UC Berkeley|
|  2|     two Taks|               0|     [250, 100]|               0|Masters|School of Informa...|UC Berkeley|
|  0|Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
+---+-------------+----------------+---------------+----------------+-------+--------------------+-----------+


AnalysisException: Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.

In [82]:
#### 해결방법 1. 다른 조인 표현식 사용
person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()

#### 해결방법 2. 조인 후 컬럼 제거
person.join(gradProgramDupe,joinExpr).drop(person["graduate_program"]).select("graduate_program").show()

#### 해결방법 3. 조인 전 컬럼 변경
grad = graduate_program.withColumnRenamed("id","grad_id")
grad_joinExpr = person["graduate_program"]==grad["grad_id"]
person.join(grad,grad_joinExpr).show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               0|
|               0|
+----------------+
+----------------+
|graduate_program|
+----------------+
|               0|
|               0|
|               0|
+----------------+
+---+-------------+----------------+---------------+-------+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status|grad_id| degree|          department|     school|
+---+-------------+----------------+---------------+-------+-------+--------------------+-----------+
|  0|Bill Chambers|               0|          [100]|      0|Masters|School of Informa...|UC Berkeley|
|  1|      one Tak|               0|[500, 250, 100]|      0|Masters|School of Informa...|UC Berkeley|
|  2|     two Taks|               0|     [250, 100]|      0|Masters|School of Informa...|UC Berkeley|
+---+-------------+----------------+---------------+-------+-------+--------------------+-----------+
